<a href="https://colab.research.google.com/github/silvererudite/simulationAndModeling/blob/main/Gender_violence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd


In [2]:
violence_data= pd.read_csv('/content/drive/MyDrive/gender_violence_dataset/violence_data.csv')
violence_data.head()

,RecordID,Country,Gender,Demographics Question,Demographics Response,Question,Survey Year,Value
0,1,Afghanistan,F,Marital status,Never married,... if she burns the food,01/01/2015,NaN
1,1,Afghanistan,F,Education,Higher,... if she burns the food,01/01/2015,10.1
2,1,Afghanistan,F,Education,Secondary,... if she burns the food,01/01/2015,13.7
3,1,Afghanistan,F,Education,Primary,... if she burns the food,01/01/2015,13.8
4,1,Afghanistan,F,Marital status,"Widowed, divorced, separated",... if she burns the food,01/01/2015,13.8


In [3]:
violence_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12600 entries, 0 to 12599
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   RecordID               12600 non-null  int64  
 1   Country                12600 non-null  object 
 2   Gender                 12600 non-null  object 
 3   Demographics Question  12600 non-null  object 
 4   Demographics Response  12600 non-null  object 
 5   Question               12600 non-null  object 
 6   Survey Year            12600 non-null  object 
 7   Value                  11187 non-null  float64
dtypes: float64(1), int64(1), object(6)
memory usage: 787.6+ KB


In [4]:
violence_data.shape

(12600, 8)

In [7]:
violence_data['Country'].value_counts()

Mali                     180
Kenya                    180
Sao Tome and Principe    180
Indonesia                180
Morocco                  180
                        ... 
Maldives                 180
Cote d'Ivoire            180
Rwanda                   180
Nicaragua                180
Zambia                   180
Name: Country, Length: 70, dtype: int64

In [8]:
violence_data['Demographics Response'].value_counts()

Employed for kind               840
Rural                           840
Married or living together      840
Widowed, divorced, separated    840
Urban                           840
Higher                          840
15-24                           840
35-49                           840
No education                    840
25-34                           840
Employed for cash               840
Primary                         840
Secondary                       840
Never married                   840
Unemployed                      840
Name: Demographics Response, dtype: int64